# Example for usage ARc Policy via a tool for a Strands Agent

Install requirements

In [ ]:
%pip install -r requirements.txt

## Setup for the agent

In [ ]:
from strands import Agent, tool
from strands.models import BedrockModel
import boto3

In [ ]:
# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ID = "<Fill in from value from the CFN Stack>"
GUARDRAIL_ID = "<Fill in from value from the CFN Stack>"
GUARDRAIL_VERSION = "DRAFT"
MODEL_ID = "anthropic.claude-3-5-sonnet-20241022-v2:0"

In [ ]:
# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    model_id=MODEL_ID,
    guardrail_id=GUARDRAIL_ID,
    guardrail_version=GUARDRAIL_VERSION,
    guardrail_trace="enabled",
)

# Create agent with the guardrail-protected model
auto_policy_agent = Agent(
    system_prompt="You are a helpful assistant that will answer questions related to auto insurance policy",
    model=bedrock_model,
)

# Use the protected agent for conversations
response = auto_policy_agent("If I have a trailer and I get into an accident is that considered a seperate vehical?")

# Handle potential guardrail interventions
if response.stop_reason == "guardrail_intervened":
    print("Content was blocked by guardrails, conversation context overwritten!")

print(f"Conversation: {json.dumps(agent.messages, indent=4)}")
